# Imports

In [62]:
import warnings
import datetime
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

# Set rules

In [69]:
MIN_ADD = datetime.timedelta(seconds = 60)
HOUR_ADD = 60 * MIN_ADD
DAY_ADD = HOUR_ADD * 24 

# Get data

In [113]:
dfMarketData = pd.read_csv('WDODATA.csv','\t')
dfMarketData.columns = ['Date','Time','Open','High','Low','Close','TickVol','Vol','Spread']
dfMarketData['Datetime'] = dfMarketData['Date'].str.replace('.','-') + ' ' + dfMarketData['Time']
dfMarketData = dfMarketData[['Datetime','Open','High','Low','Close']]
dfMarketData['Datetime'] = pd.to_datetime(dfMarketData['Datetime'])
dfMarketData = dfMarketData.set_index('Datetime')
dfMarketData = dfMarketData.sort_index()

In [114]:
dfMarketData

,Open,High,Low,Close
Datetime,,,,
2018-04-06 09:00:00,3360.0,3364.0,3360.0,3360.5
2018-04-06 09:01:00,3360.5,3361.0,3356.5,3357.0
2018-04-06 09:02:00,3357.0,3358.0,3355.0,3357.0
2018-04-06 09:03:00,3357.5,3358.5,3357.0,3357.0
2018-04-06 09:04:00,3357.5,3364.5,3357.5,3363.0
...,...,...,...,...
2023-04-06 13:55:00,5082.0,5083.0,5081.5,5082.0
2023-04-06 13:56:00,5081.5,5082.5,5081.5,5082.5
2023-04-06 13:57:00,5082.0,5084.0,5082.0,5083.5


In [115]:
dfEventsData = pd.read_csv('EventsData.csv')
dfEventsFromTo = pd.read_csv('EventsFromTo.csv')
dfEventsData['ReleaseTime'] = pd.to_datetime(dfEventsData['ReleaseTime'])
dfEventsData['PercentChng'] = dfEventsData['ActualValue']/dfEventsData['PreviousValue']-1
dfEventsData = dfEventsData.loc[dfEventsData['ReleaseTime'] > min(dfMarketData.index),:]
dfEventsData['ReleaseTime'] = pd.to_datetime(dfEventsData['ReleaseTime'])
dfEventsData = dfEventsData.set_index('ReleaseTime')
dfEventsData = dfEventsData.sort_index()

In [116]:
dfEventsData

,EventID,ActualValue,PreviousValue,Importance,PercentChng
ReleaseTime,,,,,
2018-04-06 09:30:00,840030023,102000000,287000000,2,-0.644599
2018-04-06 09:30:00,840030015,4100000,4100000,2,0.000000
2018-04-06 09:30:00,840030016,103000000,313000000,3,-0.670927
2018-04-06 09:30:00,840030018,300000,100000,2,2.000000
2018-04-06 14:00:00,840100001,808000000,797000000,2,0.013802
...,...,...,...,...,...
2023-04-03 10:45:00,840500001,49200000,49300000,2,-0.002028
2023-04-03 11:00:00,840040004,46900000,49100000,2,-0.044807
2023-04-03 11:00:00,840040001,46300000,47700000,3,-0.029350


# Split train/test

In [117]:
dfEventsDataTrain,dfEventsDataTest=np.split(dfEventsData, [int(.7*len(dfEventsData))])
dfMarketDataTrain=dfMarketData.loc[dfMarketData.index<datetime.datetime.combine(max(dfEventsDataTrain.index).date(), datetime.datetime.min.time())+DAY_ADD]
dfMarketDataTest=dfMarketData[dfMarketData.index > max(dfMarketDataTrain.index)]